In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
config = {
    'domain': 'spouse',
#     'db_name': 'babble_spouse_demo',
}

In [4]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_spouse.db


In [7]:
from snorkel import SnorkelSession
from snorkel.models import candidate_subclass

session = SnorkelSession()

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
candidate_class = Spouse

In [8]:
candidates = session.query(candidate_class).all()
print(len(candidates))

27688


In [9]:
from tutorials.babble.spouse.utils import load_external_labels

DATA_ROOT = os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/'
fpath = DATA_ROOT + 'labels.tsv'
# load_external_labels(session, Spouse, annotator_name='gold', path=fpath, splits=[1,2,3])

AnnotatorLabels created: 0
AnnotatorLabels created: 0
AnnotatorLabels created: 0


Line: candidate_class, split, candidate_id, span1, span2, gold_label, sentence
[entities specified w/ char start, char end]

In [25]:
%%time

import csv
import numpy as np
from snorkel.annotations import load_gold_labels

fpath = DATA_ROOT + 'qalf.tsv'
with open(fpath, 'wb') as csv_file:
    csvwriter = csv.writer(csv_file, delimiter='\t')
    headers = ['relation', 'split', 'candidate_id', 'span1', 'span2', 'label', 'sentence']
    csvwriter.writerow(headers)
    for split in [0, 1, 2]:
        print("Starting split {}".format(split))
        L_gold_csr = load_gold_labels(session, annotator_name='gold', split=split)
        L_gold = np.array(L_gold_csr.todense()).reshape((L_gold_csr.shape[0],))
        candidates = [L_gold_csr.get_candidate(session, i) for i in range(L_gold.shape[0])]
        for c, label in zip(candidates, L_gold):
            line = []
            line.append('Spouse')
            line.append(split)
            line.append(c.get_stable_id())
            line.append(':'.join(map(str, [c[0].char_start, c[0].char_end])))
            line.append(':'.join(map(str, [c[1].char_start, c[1].char_end])))
            line.append(label)
            line.append(c.get_parent().text.strip().encode('utf-8'))
            csvwriter.writerow(line)

CPU times: user 1min 17s, sys: 2.78 s, total: 1min 20s
Wall time: 1min 21s
